In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import pandas as pd

In [4]:
import pandas as pd

data = []

file_path = '/content/drive/MyDrive/corpus_all.txt'

with open(file_path, 'r', encoding='utf-8') as file:
    for line in file:
        parts = line.split(maxsplit=1)
        if len(parts) == 2:
            emotion, message = parts
            data.append({'emotion': emotion, 'message': message.strip()})

df = pd.DataFrame(data)

In [5]:
df.head(5)

,emotion,message
0,sad,"বরাবর, মাননীয় প্রধানমন্ত্রী গণপ্রজাতন্ত্রী বা..."
1,sad,তদন্ত করে লাভ কী ? কোন দিন কোন তদন্তের পর কিছু...
2,angry,দেশকি মধ্যম আয়ের দেশে রুপান্তর হচ্ছে নাকি মগের...
3,happy,আমি গর্বিত আমি মুসলিম। আমি সংগ্রামী। আমি যোদ্ধ...
4,angry,রাতের বেলা আবার কিসের সকাল।


In [6]:
stopwords_path ='/content/drive/MyDrive/stopwords-bn.txt'

In [7]:

with open(stopwords_path, 'r', encoding='utf-8') as file:
    stop_words = set(file.read().splitlines())

# Function to remove stop words from a message
def remove_stopwords(message):
    words = message.split()
    filtered_words = [word for word in words if word not in stop_words]
    return ' '.join(filtered_words)

# Apply the function to the 'message' column
df['message'] = df['message'].apply(remove_stopwords)

In [8]:
df.head(5)

,emotion,message
0,sad,"বরাবর, মাননীয় প্রধানমন্ত্রী গণপ্রজাতন্ত্রী বা..."
1,sad,তদন্ত লাভ ? তদন্তের হয়েছে মন ? রাজনৈতিক শক্তির...
2,angry,দেশকি মধ্যম আয়ের দেশে রুপান্তর মগের মুলুকের দে...
3,happy,গর্বিত মুসলিম। সংগ্রামী। যোদ্ধা। চির রণবীর। আল...
4,angry,রাতের বেলা কিসের সকাল।


In [9]:
df.groupby('emotion').describe()

message                                                            \
           count unique                                                top   
emotion                                                                      
angry       1416   1368                                          বিচার চাই   
disgust      703    695  অপরাধ কারী অপরাধী প্রশ্রয় দানকারী দুজনেই সমান...   
fear         384    382  হট্রগোল পরিস্থিতি ঘোলাটে নির্বাহী মেজিস্ট্রটকে...   
happy       1908   1782                                            ধন্যবাদ   
sad         1341   1301  অপরাধ কারী অপরাধী প্রশ্রয় দানকারী দুজনেই সমান...   
surprise     562    557  স্ট্যাটাসের বাকি লাইনগুলো অস্বীকার করবো ? কোনট...   

               
         freq  
emotion        
angry       8  
disgust     6  
fear        2  
happy      28  
sad        12  
surprise    4

In [10]:
df['encode'] = df['emotion'].apply(lambda x: 0 if x == 'angry' else \
                          1 if x == 'disgust' else \
                          2 if x == 'fear' else \
                          3 if x == 'happy' else \
                          4 if x == 'sad' else \
                          5 if x == 'surprise' else None)

In [11]:
df.head(4)

,emotion,message,encode
0,sad,"বরাবর, মাননীয় প্রধানমন্ত্রী গণপ্রজাতন্ত্রী বা...",4
1,sad,তদন্ত লাভ ? তদন্তের হয়েছে মন ? রাজনৈতিক শক্তির...,4
2,angry,দেশকি মধ্যম আয়ের দেশে রুপান্তর মগের মুলুকের দে...,0
3,happy,গর্বিত মুসলিম। সংগ্রামী। যোদ্ধা। চির রণবীর। আল...,3


In [12]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score, f1_score, silhouette_score, adjusted_rand_score
from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.cluster import KMeans
from transformers import BertTokenizer, TFBertModel
import tensorflow as tf

In [13]:
from transformers import AutoTokenizer, AutoModel

In [14]:
from transformers import BertTokenizer, TFBertModel
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import tensorflow as tf
import pickle

# Load tokenizer and model
tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-uncased')
model = TFBertModel.from_pretrained('bert-base-multilingual-uncased')

# Define preprocessing function with smaller max length
def preprocess_function(texts):
    return tokenizer(texts, padding=True, truncation=True, return_tensors='tf', max_length=128)

# Load and preprocess data
stratified_sample = df.groupby('encode', group_keys=False).apply(lambda x: x.sample(min(len(x), 6000))).reset_index(drop=True)
train_sample, test_sample = train_test_split(stratified_sample, train_size=5200, test_size=300, stratify=stratified_sample['encode'], random_state=42)
train_sample = train_sample.reset_index(drop=True)
test_sample = test_sample.reset_index(drop=True)
X_train, y_train = train_sample['message'], train_sample['encode']
X_test, y_test = test_sample['message'], test_sample['encode']
y_train = np.array(y_train)
y_test = np.array(y_test)
train_encodings = preprocess_function(X_train.tolist())
test_encodings = preprocess_function(X_test.tolist())

def get_bert_embeddings_in_chunks(encodings, batch_size=32):
    embeddings = []
    num_batches = (len(encodings['input_ids']) + batch_size - 1) // batch_size

    for i in range(num_batches):
        batch_encodings = {k: v[i * batch_size:(i + 1) * batch_size] for k, v in encodings.items()}
        outputs = model(**batch_encodings)
        batch_embeddings = outputs.last_hidden_state[:, 0, :].numpy()
        embeddings.append(batch_embeddings)

    return np.vstack(embeddings)

X_train_bert = get_bert_embeddings_in_chunks({'input_ids': train_encodings['input_ids'], 'attention_mask': train_encodings['attention_mask']})
X_test_bert = get_bert_embeddings_in_chunks({'input_ids': test_encodings['input_ids'], 'attention_mask': test_encodings['attention_mask']})

# Define classifiers
from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score

classifiers = {
    'Naive Bayes': GaussianNB(),
    'Decision Tree': DecisionTreeClassifier(random_state=42),
    'Random Forest': RandomForestClassifier(random_state=42),
    'Logistic Regression': LogisticRegression(random_state=42, max_iter=200),
    'SVM': SVC(kernel='rbf', C=2.0, gamma=0.6, random_state=42)
}

# Train and evaluate each classifier
for name, clf in classifiers.items():
    clf.fit(X_train_bert, y_train)
    y_train_pred = clf.predict(X_train_bert)
    y_test_pred = clf.predict(X_test_bert)

    train_accuracy = accuracy_score(y_train, y_train_pred)
    test_accuracy = accuracy_score(y_test, y_test_pred)
    f1_macro = f1_score(y_test, y_test_pred, average='macro')
    f1_micro = f1_score(y_test, y_test_pred, average='micro')
    precision = precision_score(y_test, y_test_pred, average='macro')
    recall = recall_score(y_test, y_test_pred, average='macro')

    print(f"{name}:")
    print(f"Train Accuracy: {train_accuracy:.4f}")
    print(f"Test Accuracy: {test_accuracy:.4f}")
    print(f"F1 Score (Macro): {f1_macro:.4f}")
    print(f"F1 Score (Micro): {f1_micro:.4f}")
    print(f"Precision: {precision:.4f}")
    print(f"Recall: {recall:.4f}")

    # Save the model using pickle
    model_filename = f"{name.lower().replace(' ', '_')}_model.sav"
    with open(model_filename, 'wb') as file:
        pickle.dump(clf, file)
    print(f"{name} model saved as '{model_filename}'")
    print()

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/872k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.72M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/625 [00:00<?, ?B/s]

...

Naive Bayes:
Train Accuracy: 0.3175
Test Accuracy: 0.2700
F1 Score (Macro): 0.2272
F1 Score (Micro): 0.2700
Precision: 0.2503
Recall: 0.2376
Naive Bayes model saved as 'naive_bayes_model.sav'

Decision Tree:
Train Accuracy: 0.9929
Test Accuracy: 0.2800
F1 Score (Macro): 0.1956
F1 Score (Micro): 0.2800
Precision: 0.1958
Recall: 0.1974
Decision Tree model saved as 'decision_tree_model.sav'



/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Random Forest:
Train Accuracy: 0.9929
Test Accuracy: 0.3900
F1 Score (Macro): 0.2404
F1 Score (Micro): 0.3900
Precision: 0.2699
Recall: 0.2620
Random Forest model saved as 'random_forest_model.sav'



/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


Logistic Regression:
Train Accuracy: 0.5017
Test Accuracy: 0.4067
F1 Score (Macro): 0.2557
F1 Score (Micro): 0.4067
Precision: 0.3926
Recall: 0.2751
Logistic Regression model saved as 'logistic_regression_model.sav'

SVM:
Train Accuracy: 0.7819
Test Accuracy: 0.4533
F1 Score (Macro): 0.3096
F1 Score (Micro): 0.4533
Precision: 0.3338
Recall: 0.3239
SVM model saved as 'svm_model.sav'



In [15]:
from transformers import BertTokenizer, TFBertModel
import tensorflow as tf
import numpy as np
import pickle

# Load tokenizer and model
tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-uncased')
model = TFBertModel.from_pretrained('bert-base-multilingual-uncased')

# Preprocessing function
def preprocess_function(texts):
    return tokenizer(texts, padding=True, truncation=True, return_tensors='tf', max_length=128)

# Get embeddings function
def get_bert_embeddings(encodings):
    outputs = model(**encodings)
    embeddings = outputs.last_hidden_state[:, 0, :].numpy()  # Get the embeddings for [CLS] token
    return embeddings

# Load the saved models
models = {}
model_filenames = ['naive_bayes_model.sav', 'decision_tree_model.sav', 'random_forest_model.sav', 'logistic_regression_model.sav', 'svm_model.sav']
model_names = ['Naive Bayes', 'Decision Tree', 'Random Forest', 'Logistic Regression', 'SVM']

for name, filename in zip(model_names, model_filenames):
    with open(filename, 'rb') as file:
        models[name] = pickle.load(file)

# Create a mapping dictionary
label_mapping = {
    0: 'angry',
    1: 'disgust',
    2: 'fear',
    3: 'happy',
    4: 'sad',
    5: 'surprise'
}

# Bengali sentence to predict
bengali_sentence = "আমি আজ খুব খুশি এবং আনন্দিত"  # Replace with your Bengali sentence

# Preprocess the input sentence
input_encodings = preprocess_function([bengali_sentence])

# Get embeddings for the input sentence
input_embeddings = get_bert_embeddings(input_encodings)

# Predict with each model and output the predictions
for name, model in models.items():
    prediction = model.predict(input_embeddings)
    decoded_prediction = label_mapping[prediction[0]]
    print(f"Prediction for the input sentence with {name}: {decoded_prediction}")


Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFBertModel: ['cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing TFBertModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFBertModel were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions w

Prediction for the input sentence with Naive Bayes: happy
Prediction for the input sentence with Decision Tree: disgust
Prediction for the input sentence with Random Forest: happy
Prediction for the input sentence with Logistic Regression: happy
Prediction for the input sentence with SVM: happy


In [47]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import tensorflow as tf
from transformers import BertTokenizer, TFBertModel
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score

# Load Data
# df = pd.read_csv('your_data.csv')  # Uncomment this line and replace with your data file

# Step 1: Stratified Sampling
stratified_sample = df.groupby('encode', group_keys=False).apply(lambda x: x.sample(min(len(x), 6000))).reset_index(drop=True)

# Step 2: Split the data into 5200 training and 300 testing samples
train_sample, test_sample = train_test_split(stratified_sample, train_size=5200, test_size=300, stratify=stratified_sample['encode'], random_state=42)

# Step 3: Reset Indices
train_sample = train_sample.reset_index(drop=True)
test_sample = test_sample.reset_index(drop=True)

# Step 4: Split the data into features and labels
X_train, y_train = train_sample['message'], train_sample['encode']
X_test, y_test = test_sample['message'], test_sample['encode']

# Step 5: Convert labels to NumPy arrays
y_train = np.array(y_train)
y_test = np.array(y_test)

# Load BERT tokenizer and model
tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-uncased')
bert_model = TFBertModel.from_pretrained('bert-base-multilingual-uncased')

# Define a custom layer to handle BERT encoding
class BertLayer(tf.keras.layers.Layer):
    def __init__(self, model, **kwargs):
        super(BertLayer, self).__init__(**kwargs)
        self.model = model

    def call(self, inputs):
        outputs = self.model(**inputs)
        return outputs.last_hidden_state[:, 0, :]  # Use the [CLS] token

    def get_config(self):
        config = super(BertLayer, self).get_config()
        config.update({
            "model": self.model.config.to_dict(),  # Use the model's configuration
        })
        return config

    @classmethod
    def from_config(cls, config):
        model_config = config.pop("model")
        model = TFBertModel.from_pretrained(model_config['name_or_path'])
        return cls(model, **config)

# Define preprocessing function
def preprocess_texts(texts):
    encodings = tokenizer(texts, padding='max_length', truncation=True, return_tensors='tf', max_length=128)
    return encodings

# Define a data generator
class DataGenerator(tf.keras.utils.Sequence):
    def __init__(self, texts, labels, batch_size):
        self.texts = texts
        self.labels = labels
        self.batch_size = batch_size
        self.indices = np.arange(len(self.texts))

    def __len__(self):
        return int(np.ceil(len(self.texts) / self.batch_size))

    def __getitem__(self, index):
        batch_indices = self.indices[index * self.batch_size:(index + 1) * self.batch_size]
        batch_texts = [self.texts[i] for i in batch_indices]
        batch_labels = self.labels[batch_indices]
        encodings = preprocess_texts(batch_texts)
        return {'input_ids': encodings['input_ids'], 'attention_mask': encodings['attention_mask']}, batch_labels

    def on_epoch_end(self):
        np.random.shuffle(self.indices)

# Create data generators
train_generator = DataGenerator(X_train.tolist(), y_train, batch_size=16)
test_generator = DataGenerator(X_test.tolist(), y_test, batch_size=16)

# Define the input
input_ids = tf.keras.Input(shape=(128,), dtype=tf.int32, name='input_ids')
attention_mask = tf.keras.Input(shape=(128,), dtype=tf.int32, name='attention_mask')

# Add BERT layer
bert_output = BertLayer(bert_model)({'input_ids': input_ids, 'attention_mask': attention_mask})

# Add additional layers to increase model complexity
x = tf.keras.layers.Dense(256, activation='relu')(bert_output)
x = tf.keras.layers.Dropout(0.3)(x)
x = tf.keras.layers.Dense(128, activation='relu')(x)
x = tf.keras.layers.Dropout(0.3)(x)
x = tf.keras.layers.Dense(64, activation='relu')(x)
x = tf.keras.layers.Dropout(0.3)(x)
x = tf.keras.layers.Dense(32, activation='relu')(x)
x = tf.keras.layers.Dropout(0.3)(x)

# Dynamically set the number of classes for the output layer
num_classes = len(np.unique(y_train))
output = tf.keras.layers.Dense(num_classes, activation='softmax')(x)

# Define the model
model = tf.keras.Model(inputs=[input_ids, attention_mask], outputs=[output])

# Compile the model
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-5),
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

# Fit the model using the generators and get the history
history = model.fit(train_generator, epochs=15, validation_data=test_generator)

# Save the model
model.save('my_bert_model.h5', save_format='h5')

# Evaluate the model on test data
test_loss, test_accuracy = model.evaluate(test_generator, verbose=0)
print(f"Test Accuracy: {test_accuracy:.4f}")

# Predict on test data
def predict_on_test_generator(generator):
    all_predictions = []
    all_labels = []
    for batch_inputs, batch_labels in generator:
        batch_predictions = model.predict(batch_inputs)
        all_predictions.append(np.argmax(batch_predictions, axis=1))
        all_labels.append(batch_labels)
    return np.concatenate(all_predictions), np.concatenate(all_labels)

y_pred, y_true = predict_on_test_generator(test_generator)

# Calculate metrics
f1 = f1_score(y_true, y_pred, average='weighted')
precision = precision_score(y_true, y_pred, average='weighted')
recall = recall_score(y_true, y_pred, average='weighted')

print(f"F1 Score (Test Data): {f1:.4f}")
print(f"Precision (Test Data): {precision:.4f}")
print(f"Recall (Test Data): {recall:.4f}")

# Optionally print training accuracy per epoch
train_accuracy = history.history['accuracy']
print(f"Training Accuracy per Epoch: {train_accuracy}")

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFBertModel: ['cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing TFBertModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFBertModel were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions w

Epoch 1/15


325/325 [==============================] - 197s 447ms/step - loss: 1.7644 - accuracy: 0.2221 - val_loss: 1.6926 - val_accuracy: 0.3300
Epoch 2/15
325/325 [==============================] - 135s 415ms/step - loss: 1.6733 - accuracy: 0.2848 - val_loss: 1.5546 - val_accuracy: 0.3967
Epoch 3/15
325/325 [==============================] - 135s 414ms/step - loss: 1.6046 - accuracy: 0.3325 - val_loss: 1.5209 - val_accuracy: 0.4267
Epoch 4/15
325/325 [==============================] - 135s 414ms/step - loss: 1.5215 - accuracy: 0.3873 - val_loss: 1.4817 - val_accuracy: 0.4267
Epoch 5/15
325/325 [==============================] - 137s 422ms/step - loss: 1.4232 - accuracy: 0.4471 - val_loss: 1.4662 - val_accuracy: 0.4500
Epoch 6/15
325/325 [==============================] - 135s 415ms/step - loss: 1.3334 - accuracy: 0.5008 - val_loss: 1.4463 - val_accuracy: 0.4567
Epoch 7/15
325/325 [==============================] - 135s 415ms/step - loss: 1.2230 - accuracy: 0.5417 - val_loss: 1.5540 - val_accura

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


Test Accuracy: 0.4633
1/1 [==============================] - 0s 118ms/step
F1 Score (Test Data): 0.4592
Precision (Test Data): 0.4716
Recall (Test Data): 0.4633
Training Accuracy per Epoch: [0.2221153825521469, 0.2848076820373535, 0.33250001072883606, 0.38730770349502563, 0.44711539149284363, 0.5007692575454712, 0.5417307615280151, 0.5869230628013611, 0.6273077130317688, 0.6503846049308777, 0.6786538362503052, 0.7034615278244019, 0.7234615087509155, 0.732692301273346, 0.7598077058792114]


In [48]:
import pandas as pd
import numpy as np
import tensorflow as tf
from transformers import BertTokenizer, TFBertModel
from tensorflow.keras.models import load_model

# Load Data (Uncomment and replace with your data file)
# df = pd.read_csv('your_data.csv')

# Define the custom BertLayer
class BertLayer(tf.keras.layers.Layer):
    def __init__(self, model, **kwargs):
        super(BertLayer, self).__init__(**kwargs)
        self.model = model

    def call(self, inputs):
        outputs = self.model(**inputs)
        return outputs.last_hidden_state[:, 0, :]  # Use the [CLS] token

    def get_config(self):
        config = super(BertLayer, self).get_config()
        config.update({
            "model": {
                "config": self.model.config.to_dict(),
                "name_or_path": "bert-base-multilingual-uncased"
            },
        })
        return config

    @classmethod
    def from_config(cls, config):
        model_config = config.pop("model")
        name_or_path = model_config.get('name_or_path', 'bert-base-multilingual-uncased')
        model = TFBertModel.from_pretrained(name_or_path)
        return cls(model, **config)

# Define preprocessing function
def preprocess_texts(texts):
    encodings = tokenizer(texts, padding='max_length', truncation=True, return_tensors='tf', max_length=128)
    return encodings

# Load the tokenizer and model
tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-uncased')
bert_model = TFBertModel.from_pretrained('bert-base-multilingual-uncased')

# Load the trained model
loaded_model = load_model('my_bert_model.h5', custom_objects={'BertLayer': BertLayer})

# Define a function to make predictions
def predict_with_model(texts):
    encodings_to_predict = preprocess_texts(texts)
    predict_inputs = {'input_ids': encodings_to_predict['input_ids'], 'attention_mask': encodings_to_predict['attention_mask']}
    predictions = loaded_model.predict(predict_inputs)
    predicted_labels = np.argmax(predictions, axis=1)

    # Define label mapping
    label_mapping = {
        0: 'angry',
        1: 'disgust',
        2: 'fear',
        3: 'happy',
        4: 'sad',
        5: 'surprise'
    }

    predicted_text_labels = [label_mapping[label] for label in predicted_labels]
    return predicted_text_labels

# Example usage
texts_to_predict = ["আমি আজ খুব খুশি এবং আনন্দিত"]
predicted_labels = predict_with_model(texts_to_predict)
print(predicted_labels)

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFBertModel: ['cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing TFBertModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFBertModel were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions w

1/1 [==============================] - 9s 9s/step
['happy']
